<a href="https://colab.research.google.com/github/ervasevval/TerryWinograd/blob/main/fine_tuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install jsonlines
!pip install datasets
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = '/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl'

In [ ]:
import os

klasor = "/content/drive/MyDrive/Classroom"
print("Klasördeki dosyalar:", os.listdir(klasor))

In [ ]:
print(data_path)
with open(data_path, "r", encoding="utf-8") as dosya:
    for i, satir in enumerate(dosya):
        print(f"{i+1}. satır: {satir.strip()}")
        if i >= 9:  # İlk 10 satırı yazdır
            break

In [ ]:
!pip install jsonlines
!pip install jsonline

In [ ]:
import pandas as pd
import jsonlines  # dikkat: doğru modül ismi bu

# Dosya yolu
dosya_yolu = '/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl'

# Satırları listeye al
veri = []
with jsonlines.open(dosya_yolu) as reader:
    for satir in reader:
        veri.append(satir)

veri

In [ ]:
from datasets import load_dataset

hf_dataset = load_dataset("json", data_files='/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl', split="train")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Model ve tokenizer aynı isimle uyumlu şekilde çağrılıyor
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Veri kümesi yükleniyor
dataset_path = "/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl"
hf_dataset = load_dataset("json", data_files=dataset_path, split="train")

# Tokenizasyon fonksiyonu tanımlanıyor
def tokenize_function(example):
    # prompt ve completion alanları kontrol ediliyor
    prompt = example.get("prompt", "")
    completion = example.get("completion", "")
    text = prompt + completion

    # Tokenizer uygulanıyor
    return tokenizer(text, truncation=True, padding="max_length", max_length=512)

# Tokenizasyon uygulanıyor
tokenized_dataset = hf_dataset.map(tokenize_function)


In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./hukuk-ai",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()


In [ ]:
# Modeli eval moduna alıyoruz
model.eval()  # Modeli eval moduna al

def hukuk_ai_cevapla(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

soru = (
    "Soru: Türk Medeni Kanunu'na göre resmi evlilik için yasal yaş sınırı nedir?\n"
    "Lütfen yasal dayanağı ile birlikte açıklayıcı bir şekilde cevap ver.\nCevap:"
)
cevap = hukuk_ai_cevapla(soru)
print(cevap)
